In [1]:
import optuna
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import importlib
from __imports__ import *
import data, LSTM_returns, optimization
importlib.reload(data)

<module 'data' from 'c:\\Users\\ic2594\\crypto\\AAA_Thursday1\\data\\__init__.py'>

In [3]:
import numpy as np
import pandas as pd
import ta  # pip install ta

def compute_slope(series):
    y = series.values
    x = np.arange(len(y))
    if len(y) == 0:
        return np.nan
    return np.polyfit(x, y, 1)[0]

def add_technical_features(df, look_back=14):
    df = df.copy()  
    df['Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Risk'] = df['Return'].rolling(window=look_back).std()
    df['RSI'] = ta.momentum.RSIIndicator(close=df['Close'], window=look_back).rsi()
    df['ADX'] = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=look_back).adx()
    df['MOM'] = df['Close'] - df['Close'].shift(look_back)
    df['HL'] = df['High'] - df['Low']
    df['HO'] = df['High'] - df['Open']
    df['LO'] = df['Low'] - df['Open']
    df['buy_pressure_ratio'] = df['Taker buy quote asset volume'] / df['Quote asset volume']
    df['trades_per_volume'] = df['Number of trades'] / df['Quote asset volume']
    df['slope'] = df['Close'].rolling(window=look_back).apply(compute_slope, raw=False)
    df = df.dropna()
    cols_to_drop = [
        'Open', 'High', 'Low', 'Quote asset volume',
        'Number of trades', 'Taker buy base asset volume',
        'Taker buy quote asset volume', 'F&G'
    ]
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])
    return df

def prep_data(df, sequence_length=60, test_size=0.2):
    # 1) include 'Close' since that's what we'll predict
    features = [
        'Close',      # ← our prediction target
        'Volume', 'F&G category', 'Return', 'Risk', 'RSI', 'ADX',
        'MOM', 'HL', 'HO', 'LO',
        'buy_pressure_ratio', 'trades_per_volume', 'slope'
    ]
    
    num_feats = [f for f in features if f != 'F&G category']
    df_num = df[num_feats]
    df_cat = pd.get_dummies(df['F&G category'], prefix='F_G')
    
    df_all = pd.concat([df_num, df_cat], axis=1).dropna()
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df_all.values)
    feature_names = df_all.columns.tolist()
    
    target_idx = feature_names.index('Close')
    X, y = [], []
    for i in range(sequence_length, len(data_scaled)):
        X.append(data_scaled[i-sequence_length:i])
        # y is the scaled Close price at time i (next day)
        y.append(data_scaled[i, target_idx])
    X, y = np.array(X), np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, shuffle=False
    )
    return (X_train, X_test, y_train, y_test), scaler


In [4]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ——— Hyperparameters (from Optuna) ———
UNITS_L0      = 160
DROPOUT_L0    = 0.0
LEARNING_RATE = 0.0007280355873484089
OPTIMIZER     = Adam(learning_rate=LEARNING_RATE)
BATCH_SIZE    = 32
MAX_EPOCHS    = 50

def train_lstm_for_asset(df, sequence_length=30, test_size=0.2):
    # … prep_data as before …
    (X_train, X_val, y_train, y_val), scaler = prep_data(
        df, sequence_length=sequence_length, test_size=test_size
    )
    
    # Build the model
    model = Sequential([
        Input(shape=X_train.shape[1:]),
        LSTM(UNITS_L0, return_sequences=False, dropout=DROPOUT_L0),
        Dense(1, activation='linear')
    ])
    
    # *** instantiate a new optimizer here, not reuse a global one ***
    optimizer = Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Train
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=MAX_EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es],
        verbose=1
    )
    
    return model, scaler, history, (X_val, y_val)

# Example usage for any asset DataFrame `asset_df`:
# model, scaler, history, (X_val, y_val) = train_lstm_for_asset(asset_df)

# And to plot predictions vs actual:
# preds = model.predict(X_val).flatten()
# plt.figure(figsize=(10,5))
# plt.plot(y_val,  label='Actual Close (scaled)')
# plt.plot(preds, label='Predicted Close (scaled)')
# plt.legend()
# plt.title("LSTM Forecast (Scaled)")
# plt.show()

In [5]:
import os
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import load_model

# 1. Ensure eager mode for compatibility (if you set run_eagerly earlier)
tf.config.run_functions_eagerly(True)

# 2. Directories to save models and scalers
os.makedirs('models', exist_ok=True)
os.makedirs('scalers', exist_ok=True)

# 3. List of asset tickers
assets = ['SOLUSDT', 'BTCUSDT', 'ETHUSDT', 'DOGEUSDT', 'XRPUSDT']
results = {}

for ticker in assets:
    # 4a) Load & feature-engineer
    df = data.load_asset(ticker, sampling='1d')
    df = data.add_fear_and_greed(df)
    df = add_technical_features(df)
    df = df[(df.index >= '2022-01-01') & (df.index <= '2023-12-31')]

    # 4b) Train the LSTM
    model, scaler, history, (X_val, y_val) = train_lstm_for_asset(df)

    # 4c) Save model and scaler
    model_path  = f'models/{ticker}_lstm_model.h5'
    scaler_path = f'scalers/{ticker}_scaler.pkl'
    model.save(model_path)
    joblib.dump(scaler, scaler_path)

    # 4d) Store paths and validation data
    results[ticker] = {
        'model_path':  model_path,
        'scaler_path': scaler_path,
        'history':     history,
        'X_val':       X_val,
        'y_val':       y_val
    }

# 5. Compute and print validation RMSE for each asset
for ticker, res in results.items():
    # Load model (if not in memory) and validation data
    model = load_model(res['model_path'])
    X_val = res['X_val']
    y_val = res['y_val']

    # Predict and score
    preds = model.predict(X_val).flatten()
    rmse  = np.sqrt(mean_squared_error(y_val, preds))
    print(f"{ticker} validation RMSE (scaled): {rmse:.4f}")


Epoch 1/50


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 248ms/step - loss: 0.0254 - val_loss: 0.0065
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 231ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 231ms/step - loss: 0.0011 - val_loss: 0.0032
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 261ms/step - loss: 9.3666e-04 - val_loss: 0.0037
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 266ms/step - loss: 5.6086e-04 - val_loss: 0.0042
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 245ms/step - loss: 7.2698e-04 - val_loss: 0.0032
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 227ms/step - loss: 4.8466e-04 - val_loss: 0.0035
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 234ms/step - loss: 4.2544e-04 - val_loss: 0.0031
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 237ms/step - loss: 3.7251e-04 - val_loss: 0.0029
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 245ms/step - loss: 2.9852e-04 - val_loss: 0.0032
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 249ms/step - loss: 4.2447e-04 - val_loss: 0.0027
Epoch 12/50
18/18 ━━━━━━━━━━━━━

Epoch 1/50


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


...

c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step


DOGEUSDT validation RMSE (scaled): 0.0265
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step

c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step
XRPUSDT validation RMSE (scaled): 0.0453


In [6]:
test_predictions = {}

for ticker, res in results.items():
    model = load_model(res['model_path'])
    # Predict on the test set
    preds = model.predict(res['X_val']).flatten()
    dates = df.index[-len(preds):]  # Get the corresponding dates
    test_predictions[ticker] = pd.Series(preds, index=dates)

# Now `test_predictions` contains the predictions for each ticker indexed by date
%store test_predictions
joblib.dump(test_predictions, "models/test_predictions.pkl")

1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step

c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Stored 'test_predictions' (dict)


['models/test_predictions.pkl']